In [121]:
from numpy.random import randn
import numpy as np
import pandas as pd
np.random.seed(123)
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.lines import Line2D 
import math
import plotly
plotly.tools.set_credentials_file(username='koryakinp', api_key='rxhJKQ4fAbxfoeYCPUxz')
import plotly.plotly as py
import plotly.graph_objs as go

plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4)
pd.options.display.max_rows = 10

def get_bearing(origin,destination):
    startLat = math.radians(origin['latitude'])
    startLong = math.radians(origin['longtitude'])
    endLat = math.radians(destination['latitude'])
    endLong = math.radians(destination['longtitude'])

    dLong = endLong - startLong

    dPhi = math.log(math.tan(endLat/2.0+math.pi/4.0)/math.tan(startLat/2.0+math.pi/4.0))
    if abs(dLong) > math.pi:
        if dLong > 0.0:
            dLong = -(2.0 * math.pi - dLong)
        else:
            dLong = (2.0 * math.pi + dLong)

    return (math.degrees(math.atan2(dLong, dPhi)) + 360.0) % 360.0;

In [122]:
flights = pd.read_csv('data/flights.csv', low_memory=False, usecols=['ORIGIN_AIRPORT','DESTINATION_AIRPORT','ARRIVAL_DELAY'])
airports = pd.read_csv('data/airports.csv', low_memory=False, usecols=['IATA_CODE','LATITUDE','LONGITUDE'])

In [123]:
flights = flights.merge(airports, how = 'left', left_on = 'ORIGIN_AIRPORT', right_on = 'IATA_CODE')
flights = flights.merge(airports, how = 'left', left_on = 'DESTINATION_AIRPORT', right_on = 'IATA_CODE')
flights.rename(columns={
    'LATITUDE_x': 'O_LATITUDE', 
    'LONGITUDE_x': 'O_LONGTITUDE', 
    'LATITUDE_y': 'D_LATITUDE', 
    'LONGITUDE_y': 'D_LONGTITUDE'}, inplace=True)
flights.drop('IATA_CODE_x',axis=1, inplace=True)
flights.drop('IATA_CODE_y',axis=1, inplace=True)

In [124]:
flights

,ORIGIN_AIRPORT,DESTINATION_AIRPORT,ARRIVAL_DELAY,O_LATITUDE,O_LONGTITUDE,D_LATITUDE,D_LONGTITUDE
0,ANC,SEA,-22.0,61.17432,-149.99619,47.44898,-122.30931
1,LAX,PBI,-9.0,33.94254,-118.40807,26.68316,-80.09559
2,SFO,CLT,5.0,37.61900,-122.37484,35.21401,-80.94313
3,LAX,MIA,-9.0,33.94254,-118.40807,25.79325,-80.29056
4,SEA,ANC,-21.0,47.44898,-122.30931,61.17432,-149.99619
...,...,...,...,...,...,...,...
5819074,LAX,BOS,-26.0,33.94254,-118.40807,42.36435,-71.00518
5819075,JFK,PSE,-16.0,40.63975,-73.77893,18.00830,-66.56301
5819076,JFK,SJU,-8.0,40.63975,-73.77893,18.43942,-66.00183
5819077,MCO,SJU,-10.0,28.42889,-81.31603,18.43942,-66.00183


In [125]:
flights['BEARING'] = flights.apply(lambda row: get_bearing({
    'latitude':row['O_LATITUDE'],
    'longtitude':row['O_LONGTITUDE']},{
    'latitude':row['D_LATITUDE'],
    'longtitude':row['D_LONGTITUDE']}), axis=1)
    

In [177]:
copy = flights.copy(deep=True)
copy.drop('ORIGIN_AIRPORT',axis=1, inplace=True)
copy.drop('DESTINATION_AIRPORT',axis=1, inplace=True)
copy.drop('O_LATITUDE',axis=1, inplace=True)
copy.drop('O_LONGTITUDE',axis=1, inplace=True)
copy.drop('D_LATITUDE',axis=1, inplace=True)
copy.drop('D_LONGTITUDE',axis=1, inplace=True)

trace1 = go.Area(
    r=copy.groupby(pd.cut(copy["BEARING"], np.arange(0, 365, 5)), as_index=False).mean()['ARRIVAL_DELAY'],
    t=np.arange(0, 365, 5),
    name = 'Mean Delay, min'
)

layout = go.Layout(title='Delay vs Bearing')
fig = go.Figure(data = [trace1], layout=layout)
py.iplot(fig, filename='polar-area-chart')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~koryakinp/0 or inside your plot.ly account where it is named 'polar-area-chart'
